In [1]:
import ipynb
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

from util import *

import xgboost as xgb
from xgboost import XGBRegressor

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import zipfile
from io import BytesIO
import pickle

In [2]:
# Load data:
path_1 = 'C:/Users/sjurg/OneDrive/Documents/Python/misc_data_sets/sberbank-russian-housing-market/'
path_2 = 'C:/Users/sjurg/OneDrive/Documents/Python/DS-ML_Projects/house_pricing/extracted/'
path_3 = 'C:/Users/sjurg/OneDrive/Documents/Python/DS-ML_Projects/house_pricing/preds/'
macro = zipped_csv_to_df(path_1 + "macro.csv.zip")
Train = zipped_csv_to_df(path_1 + "train.csv.zip")
Test = zipped_csv_to_df(path_1 + "test.csv.zip")

# Store id's separately:
train_id = Train['id']
test_id = Test['id']
# Merge the train and macro feature data sets based on timestamps; remove the target column:
combined_train = pd.merge_ordered(Train, macro, on='timestamp', how='left').drop('id', axis=1)
# Merge the test and macro feature data sets based on timestamps:
combined_test = pd.merge_ordered(Test, macro, on='timestamp', how='left').drop('id', axis=1)
# Create train and test target sets:
target_train = Train['price_doc']

# Get 100 most important features from data exploration:
with open(path_2 + "top_100", "rb") as fp:
    top_100 = pickle.load(fp)
top_100.remove('timestamp')

In [3]:
combined_train_clean = clean_level_1(combined_train)
combined_train_clean = clean_level_2(combined_train_clean)
combined_train_clean = add_dates(combined_train_clean)
combined_train_clean = add_misc_features(combined_train_clean)
combined_train_clean = ordinal_encoding(combined_train_clean)
train_target = combined_train_clean['price_doc']
combined_train_clean = combined_train_clean.drop('price_doc', axis=1)[top_100]

combined_test_clean = clean_level_1(combined_test)
combined_test_clean = clean_level_2(combined_test_clean, train=False)
combined_test_clean = add_dates(combined_test_clean)
combined_test_clean = add_misc_features(combined_test_clean)
combined_test_clean = clean_train_set_only(combined_test_clean)
combined_test_clean = ordinal_encoding(combined_test_clean)[top_100]

In [4]:
model = XGBRegressor(n_estimators=200, eta=0.1, max_depth=2, subsample=1.0, colsample_bytree=0.7, objective='reg:squarederror',
                    eval_metric='rmse')
model.fit(combined_train_clean, np.log1p(train_target))

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric='rmse', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=200, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0, ...)

In [5]:
test_predictions = model.predict(combined_test_clean)
test_predictions = np.expm1(test_predictions)

In [6]:
predictions = pd.DataFrame({'id': test_id, 'price_doc': test_predictions})
predictions.to_csv(path_3 + "predictions2.csv", index=False)

This produces a score of 0.32793. Marginal improvement compared to the more 'basic' data preparation...